# Track vehicles
Use trained classifier on videostream in addition to the lange finding pipeline

In [1]:
#calibration

#find corner points in calibration images using CV2 as in lesson
%matplotlib inline
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import os

#set chessboard size for calib images
nx = 9
ny = 6
imagepointlist = []
objectpointlist = []

#load all calibration images
cwd = os.getcwd()
images = glob.glob(cwd+ '/camera_cal/calibration*.jpg')

#array with x,y,z coords for one image
objectpoints = np.zeros((ny*nx,3), np.float32)
objectpoints[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
for fname in images:
    img = mpimg.imread(fname)
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)

    # If found, append both sets of coordinates to lists
    if ret == True:
        imagepointlist.append(corners)
        objectpointlist.append(objectpoints)
        
#Calibrate and undistort 
img = cv2.imread(cwd+ '/camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

#calibrate with object and imagepoints
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objectpointlist, imagepointlist, img_size,None,None)


In [2]:
#pipeline
import collections


def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def undist(img):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst


def select_yellow(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    lower = np.array([20,60,60])
    upper = np.array([38,174, 250])
    mask = cv2.inRange(hsv, lower, upper)
    return mask

def select_white(image):
    lower = np.array([202,202,202])
    upper = np.array([255,255,255])
    mask = cv2.inRange(image, lower, upper)
    return mask

def combined_thres(img, s_thresh=(90, 255),h_thresh=(15, 100), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    
    h_binary = np.zeros_like(h_channel)
    h_binary[(h_channel >= h_thresh[0]) & (h_channel <= h_thresh[1])] = 1

    
    color_binary = np.zeros_like(s_binary)
    color_binary[(s_binary > 0) | (sxbinary > 0)] = 1
    
    
    return color_binary


def persp_transform(img):
    img_size = (img.shape[1], img.shape[0])
    #source and destination points eye-balled from first test picture
    offset = 100
    src = np.float32([[707,465], [1095, 720] ,[208,720],[576, 465] ])
    dst = np.float32([[950,0], [950, 720] ,[280, 720],[280, 0] ])
    a1 = [offset, offset] #upper left
    b1 = [img_size[0]-offset, offset] #upper right
    c1 = [img_size[0]-offset, img_size[1]-offset] # lower right
    d1 = [offset, img_size[1]-offset] # lower left
    #dst = np.float32([b1,c1,d1,a1])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    img_topd = cv2.warpPerspective(img, M, img_size)
   
    return M, Minv, img_topd


def sliding_win_search(img):
    #binary_warped = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    binary_warped = img

    histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high), (0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high), (0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    if leftx.any() and lefty.any() and rightx.any() and righty.any():
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
    else:
        left_fit = [0,0,0]
        right_fit = [0,0,0]

    return left_fit, right_fit, leftx, lefty, rightx, righty

def update_search(img, left_fit, right_fit):
    
    #shorter search as shown in lessons
    #binary_warped = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    binary_warped = img

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
    left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
    left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
    right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
    right_fit[1]*nonzeroy + right_fit[2] + margin)))  

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    # Generate x and y values for plotting
    #ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    #left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    #right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fit, right_fit, leftx, lefty, rightx, righty



def draw_lines(img, img_undist, Minv, left_fit, right_fit):
    #draw result
    # Create an image to draw the lines on
    #binary_warped = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    binary_warped = img

        
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    #points for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (img.shape[1], img.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(img_undist, 1, newwarp, 0.3, 0)
    return result



# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.recent_polies = []
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = 0 
        #distance in meters of vehicle center from the line
        self.line_base_posx = 0 
        self.line_base_pos = 0 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    def update_with_fit(self, current_poly):
        #update attributes for sanity check
        self.diffs = abs(np.subtract(self.current_fit,current_poly))
        self.current_fit = current_poly
        
        #distance in meters from bottom fitted x value to lane center
        ploty = 720
        basex = self.current_fit[0]*ploty**2 + self.current_fit[1]*ploty + self.current_fit[2]
        self.line_base_pos = abs((1280/2)-basex) * 3.7/700
        self.line_base_posx = basex
        
        if len(self.recent_polies)>4:
            self.recent_polies.pop(0)
            self.recent_polies.append(current_poly)
            self.best_fit = np.mean(self.recent_polies, axis=0)
        else:
            self.recent_polies.append(current_poly)
            self.best_fit = np.mean(self.recent_polies, axis=0)

    def get_curvature(self):
        #update curvature
        x = self.allx
        y = self.ally
        if x.any() and y.any():
            ploty = np.linspace(0, 719, num=720)# to cover same y-range as image

            y_eval = np.max(ploty)
            ym_per_pix = 30/720 # meters per pixel in y dimension
            xm_per_pix = 3.7/700 # meters per pixel in x dimension

            # Fit new polynomials to x,y in world space
            fit_cr = np.polyfit(y*ym_per_pix, x*xm_per_pix, 2)
            # Calculate the new radii of curvature
            self.radius_of_curvature = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
        else:
            self.radius_of_curvature = 0

def lines_checkout(left,right):
    #Checking that they have similar curvature
    tol = 0.1
    
    if np.isclose(left.radius_of_curvature,right.radius_of_curvature, rtol = tol):
        pass
    else:
        return False

    #Checking that they are separated by approximately the right distance horizontally
    separation = abs(left.line_base_pos) + abs(right.line_base_pos)
    if np.isclose(separation, 3.7, rtol = tol):
    #if np.isclose(left.line_base_pos,(3.7/2), rtol = tol) and np.isclose(right.line_base_pos,(3.7/2), rtol = tol):
        pass
    else:
        return False
    
    #Checking that they are roughly parallel
    if np.isclose(left.current_fit,right.current_fit, rtol = tol).all():
        pass
    else:
        return False

    return True

def init_globals():
    #for caching values
    global left_lane
    global right_lane
    global stale
    global tracked_cars
    global heatmaps
    global counter
    counter = 0
    stale = 0
    left_lane = Line()
    right_lane = Line()
    tracked_cars = Vehicles()
    heatmaps = collections.deque(maxlen=20)

def dist_from_center(left, right):
    #distanec of vehicle center to center of detected lane in m
    x1 = left.line_base_posx
    x2 = left.line_base_posx
    centerlane = (x1+x2)/2 + x1
    dist = abs((1280/2) - centerlane) * 3.7/700
    return dist
    


In [3]:
#new functions for vehicle detection and tracking
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from scipy.ndimage.measurements import label

#color features
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector

    return hist_features
#spatial features
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features

#utility function for find_cars
def convert_color(img, conv='RGB2YCrCb'):
    if conv == 'RGB2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    if conv == 'BGR2YCrCb':
        return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    if conv == 'RGB2LUV':
        return cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
    
# Define a single function that can extract features using hog sub-sampling and make predictions
def find_cars(img, ystart, ystop,xstart, xstop, scale, svc, X_scaler, orient, pix_per_cell, cell_per_block, spatial_size, hist_bins):
    bounding_box_list = []
    draw_img = np.copy(img)
    img = img.astype(np.float32)/255

    img_tosearch = img[ystart:ystop,xstart:xstop,:]
    ctrans_tosearch = convert_color(img_tosearch, conv='RGB2YCrCb')
    if scale != 1:
        imshape = ctrans_tosearch.shape
        ctrans_tosearch = cv2.resize(ctrans_tosearch, (np.int(imshape[1]/scale), np.int(imshape[0]/scale)))
        
    ch1 = ctrans_tosearch[:,:,0]
    ch2 = ctrans_tosearch[:,:,1]
    ch3 = ctrans_tosearch[:,:,2]

    # Define blocks and steps as above
    nxblocks = (ch1.shape[1] // pix_per_cell) - cell_per_block + 1
    nyblocks = (ch1.shape[0] // pix_per_cell) - cell_per_block + 1 
    nfeat_per_block = orient*cell_per_block**2
    
    # 64 was the orginal sampling rate, with 8 cells and 8 pix per cell
    window = 64
    nblocks_per_window = (window // pix_per_cell) - cell_per_block + 1
    cells_per_step = 2  # Instead of overlap, define how many cells to step
    nxsteps = (nxblocks - nblocks_per_window) // cells_per_step
    nysteps = (nyblocks - nblocks_per_window) // cells_per_step
    
    # Compute individual channel HOG features for the entire image
    hog1 = get_hog_features(ch1, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog2 = get_hog_features(ch2, orient, pix_per_cell, cell_per_block, feature_vec=False)
    hog3 = get_hog_features(ch3, orient, pix_per_cell, cell_per_block, feature_vec=False)
    
    for xb in range(nxsteps):
        for yb in range(nysteps):
            ypos = yb*cells_per_step
            xpos = xb*cells_per_step
            # Extract HOG for this patch
            hog_feat1 = hog1[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat2 = hog2[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_feat3 = hog3[ypos:ypos+nblocks_per_window, xpos:xpos+nblocks_per_window].ravel() 
            hog_features = np.hstack((hog_feat1, hog_feat2, hog_feat3))

            xleft = xpos*pix_per_cell
            ytop = ypos*pix_per_cell

            # Extract the image patch
            subimg = cv2.resize(ctrans_tosearch[ytop:ytop+window, xleft:xleft+window], (64,64))
          
            # Get color features
            spatial_features = bin_spatial(subimg, size=spatial_size)
            hist_features = color_hist(subimg, nbins=hist_bins)

            # Scale features and make a prediction

            test_features = X_scaler.transform(np.hstack((spatial_features, hist_features, hog_features)).reshape(1, -1))
            #test_prediction = svc.predict(test_features)
            test_prediction = svc.predict_proba(test_features)

            #if test_prediction == 1:
            if test_prediction[0][1] > 0.9:
                xbox_left = np.int(xleft*scale)
                ytop_draw = np.int(ytop*scale)
                win_draw = np.int(window*scale)
                #cv2.rectangle(draw_img,(xbox_left, ytop_draw+ystart),(xbox_left+win_draw,ytop_draw+win_draw+ystart),(0,0,255),6) 
                bounding_box_list.append(((xbox_left+xstart, ytop_draw+ystart),(xbox_left+win_draw+xstart,ytop_draw+win_draw+ystart)))
                
    #return draw_img, bounding_box_list
    return bounding_box_list


def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    bblist = []
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

def get_label_bbs(labels):
    #find the aggregated bounding boxes
    goodbblist = []
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        goodbblist.append(bbox)
    return goodbblist

def draw_bboxes(img, bblist):
    # Iterate through all detected cars
    for bbox in bblist:
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 7)
    # Return the image
    return img

# Define a class to receive the characteristics of each line detection
class Vehicles():
    def __init__(self):
        
        self.recent_carbbs = []
        #list of bounding boxes of cars detected in last n frames
        self.historicalbbs = []
        #last heatmaps
        self.heatlist = []
        
    def get_current_heat(self):
        #not used as still buggy
        res = self.heatlist[0]
        for i in self.heatlist[1:]:
            res += i
        self.currentheatmap = res
        
    def updateheat(self, newheatmap):
        #not used as still buggy
        #save last 5 heatmaps
        if len(self.heatlist) > 9:
            #remove oldest if list too long
            self.heatlist.pop(0)
        self.heatlist.append(newheatmap)

def check_box(newbb, oldbblist):
    #check new bounding box for distance to last bbs
    #sample box ((1128, 448), (1151, 495))
    currentx = (newbb[0][0] + newbb[1][0])/2
    currenty = (newbb[0][1] + newbb[1][1])/2

    for bb in oldbblist:
        oldx = (bb[0][0] + bb[1][0])/2
        oldy = (bb[0][1] + bb[1][1])/2
        distance = np.sqrt((abs(oldx-currentx))**2 + (abs(oldy-currenty))**2)
        #arbitrary value of permissable pixel distance (should be weighted by y-position)
        if distance < 250:
            return True
    return False
        
        
def check_bbs(goodbblist, oldbblist):
    leftoverbbs = []
    for bb in goodbblist:
        if check_box(bb, oldbblist):
            leftoverbbs.append(bb)
    return leftoverbbs
        
            


In [5]:

import pickle

# load dumped SVM classifier and scaler

with open('svm_class.pkl', 'rb') as fid:
    svc = pickle.load(fid)

with open('xscaler.pkl', 'rb') as fid:
    X_scaler = pickle.load(fid)

def process_image(img):
    global stale
    global left_lane
    global right_lane
    global tracked_cars
    global heatmaps
    global counter
    
    #########################################
    ### lane marking from previous project###

    img_undist = undist(img)
    
    mask_y = select_yellow(img_undist)
    mask_w = select_white(img_undist)
    img_yselect = cv2.bitwise_and(img_undist,img_undist,mask = mask_y)
    img_wselect = cv2.bitwise_and(img_undist,img_undist,mask = mask_w)
    img_cselect = cv2.addWeighted(img_yselect, 1., img_wselect, 1., 0.)

    img_thres = combined_thres(img_cselect)
    M, Minv, img_warp = persp_transform(img_thres)
    
    #reduce to region of interest
    roi_y_limit = 0.63
    x_offset = 0.026
    x_hor_width = 0.042
    
    imshape = img_thres.shape
    vertices = np.array([[((imshape[1] * x_offset),imshape[0]),
                          ((imshape[1]/2.0) - int(round(imshape[1] * x_hor_width)), int(round(roi_y_limit * imshape[0]))),
                          ((imshape[1]/2.0) + int(round(imshape[1] * x_hor_width)), int(round(roi_y_limit * imshape[0]))),
                          ((imshape[1] - int(round(imshape[1] * x_offset)), imshape[0])) 
                         ]], dtype=np.int32)
    img_thres = region_of_interest(img_thres, vertices)
    
    
    #if data is stale start new search, otherwise update search
    if stale == 0 or stale > 4:
        #start new sliding window search
        left_poly, right_poly, left_lane.allx, left_lane.ally, right_lane.allx, right_lane.ally = sliding_win_search(img_warp)
    else:
        #update window search
        left_poly, right_poly, left_lane.allx, left_lane.ally, right_lane.allx, right_lane.ally = update_search(img_warp, left_lane.current_fit, right_lane.current_fit)

    
    if lines_checkout(left_lane, right_lane) or stale == 0:
    #update lines if result is good or initial frame
        left_lane.update_with_fit(left_poly)
        right_lane.update_with_fit(right_poly)
        left_lane.get_curvature()
        right_lane.get_curvature()
        #img_final = draw_lines(img_warp, img_undist, Minv, left_lane.current_fit, right_lane.current_fit)
        img_final = draw_lines(img_warp, img_undist, Minv, left_lane.best_fit, right_lane.best_fit)
    else:
        #draw with average of last several fits
        img_final = draw_lines(img_warp, img_undist, Minv, left_lane.best_fit, right_lane.best_fit)
        #increase counter for stale lines
        stale += 1
    #add annotation to image
    dist_center = dist_from_center(left_lane, right_lane)
    img_stringl = 'Curvature left:  {}'.format(left_lane.radius_of_curvature)
    img_stringr = 'Curvature right:  {}'.format(right_lane.radius_of_curvature)
    img_stringd = 'Distance from center:  {}'.format(dist_center)
    
    img_final = cv2.putText(img_final, img_stringl, (15,15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255)
    img_final = cv2.putText(img_final, img_stringr, (15,35), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255)
    img_final = cv2.putText(img_final, img_stringd, (15,55), cv2.FONT_HERSHEY_SIMPLEX, 0.5, 255)
    

    
    #########################################
    #add tracking functionality 
    #########################################
    #sliding window search
    ystart = 400
    ystop = 656
    xstart = 300
    xstop = 1280
    pix_per_cell = 8
    cell_per_block = 2
    hist_bins=32
    hist_range=(0, 256)
    #colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
    orient = 9
    #hog_channel = 'ALL' # Can be 0, 1, 2, or "ALL"
    sp_size=(32, 32)
    
    #scales = [1,1.5, 2]
    scales = [1.1,1.4, 1.8, 2.4, 2.9, 3.4]
    big_box_list = []
    for sc in scales:
        if sc <2.5 and sc > 1.8:
            ystart = 420
            ystop = 656
        if sc > 2.5:
            ystart = 450
            ystop = 656
        box_list = find_cars(img, ystart, ystop,xstart,xstop, sc, svc, X_scaler, orient, pix_per_cell, cell_per_block, sp_size, hist_bins)
        big_box_list.extend(box_list)
    
    #heatmap for duplicate and false positive removal
    heat = np.zeros_like(img[:,:,0]).astype(np.float)
    heat_map = add_heat(heat,big_box_list)
    heatmaps.append(heat_map)
    heatmap_sum = sum(heatmaps)
    thresh_heat = apply_threshold(heatmap_sum, 14)
    labels = label(thresh_heat)

    
    goodbblist = get_label_bbs(labels)

    #check distance to existing boxes unless first frame, no boxe
    #save boxes without checking every 0.5 seconds to reset starting point for distance
    if len(tracked_cars.recent_carbbs)>0 and (counter % 15 == 0):
        leftoverbbs = check_bbs(goodbblist, tracked_cars.recent_carbbs)
        #leftoverbbs = goodbblist
    else:
        leftoverbbs = goodbblist
    tracked_cars.recent_carbbs = leftoverbbs
    img_final = draw_bboxes(img_final, leftoverbbs)
    counter += 1
    return img_final



In [8]:
#render video
from moviepy.editor import VideoFileClip
from IPython.display import HTML

init_globals()
#white_output = 'p_output_lanelines.mp4'
white_output = 'test_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
#clip1 = VideoFileClip("test_video.mp4")
#clip1 = VideoFileClip("project_video.mp4").subclip(40,43)
#clip1 = VideoFileClip("project_video.mp4").subclip(0,3)
clip1 = VideoFileClip("project_video.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_output.mp4
[MoviePy] Writing video test_output.mp4


100%|█████████████████████████████████████▉| 1260/1261 [31:35<00:01,  1.53s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_output.mp4 

Wall time: 31min 36s
